---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [5]:
import re
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)

doc = []
for i in range(1, len(df)):
    doc.append(re.sub(r'[,|(|)|.]', '', df[i]))
    
df = pd.Series(doc)

In [130]:
table1 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})').reset_index()
table2 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})').reset_index()
table3_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b').reset_index()
table3_2 = df.str.extractall(r'(\d{1,2})*[/-]*(\d{1,2})[/-](\d{4})\b').reset_index()
table4_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]').reset_index()
table4_2 = df.str.extractall(r'^(\d{4})[^0-9]').reset_index()
table4 = pd.concat([table4_1, table4_2])

In [131]:
table4_new = pd.Series() 

table4_new = table4.merge(table1, how='left', on = 'level_0')
table4_new1 = table4_new.iloc[:,0:3][pd.isnull(table4_new['0_y'])]

table4_new = table4_new1.merge(table2, how='left', on = 'level_0')
table4_new2 = table4_new.iloc[:,0:3][pd.isnull(table4_new['match'])]

table4_new = table4_new2.merge(table3, how='left', on = 'level_0')
table4_new3 = table4_new.iloc[:,0:3][pd.isnull(table4_new['match'])]
table4 = table4_new3

In [132]:
table1['day'] = table1[1].apply(lambda x:x.split(' ')[0])
table1['year'] = table1[1].apply(lambda x:x.split(' ')[1])
table1.columns=['level_0','match','month','day-year','day','year']
table1.drop('day-year',axis=1,inplace=True)

table1 = table1[['level_0', 'match', 'day', 'month', 'year']]

In [133]:
table2.columns=['level_0','match','day','month','year']
table2['day'] = table2['day'].replace(np.nan,1)

In [ ]:
day month year

In [134]:
table3_1.columns=['level_0','match','month','day','year']
table3_1['year']=table3_1['year'].apply(str)
table3_1['year']=table3_1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)

table3_1 = table3_1[['level_0', 'match', 'day', 'month', 'year']]

table3_2.columns=['level_0', 'match', 'day', 'month', 'year']
table3_2['day'] = table3_2['day'].replace(np.nan,1)

In [135]:
table4.columns=['level_0','match','year']
table4['month']=1
table4['day']=1
table4 = table4[['level_0', 'match', 'day', 'month', 'year']]

In [136]:
table_fin = pd.concat([table1, table2, table3_1, table3_2, table4])

In [139]:
table_fin['month'] = table_fin['month'].str.extract('(\w{3}|\d{1,2})').str.strip()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [ ]:
final['fin_date'] = pd.to_datetime(table_fin['month'].apply(str)+'/'+table_fin['day'].apply(str)+'/'+table_fin['year'].apply(str), format='%B/%d/%Y')

In [3]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here